In [12]:
"""
renko_backtest_groww_margin_intraday_eod.py

Intraday Renko strategy:
- 15m data, ATR(14) Renko
- RSI14 & StochRSI(14) cross at Renko S/R
- Groww intraday fees
- 1L equity capital per symbol, 5x margin (max exposure = 5L)
- Forced square-off at end of EACH trading day (no overnight holds)
- All timestamps converted to IST
"""

import math
import logging
from dataclasses import dataclass
from typing import List, Tuple, Optional

import numpy as np
import pandas as pd
import yfinance as yf

# ======================= CONFIG =======================

@dataclass
class Config:
    tickers: List[str] = None
    period: str = "60d"
    interval: str = "15m"
    rsi_len: int = 14
    stoch_len: int = 14
    atr_len: int = 14
    sr_tolerance: float = 0.25
    initial_capital: float = 100_000.0  # equity per symbol
    leverage: float = 5.0               # intraday margin
    out_trades_csv: str = "trades.csv"


CFG = Config(
    tickers=['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS',
             'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS',
             'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'EICHERMOT.NS',
             'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS',
             'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS', 'INFY.NS',
             'INDIGO.NS', 'JSWSTEEL.NS', 'JIOFIN.NS', 'KOTAKBANK.NS', 'LT.NS',
             'M&M.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'NTPC.NS', 'NESTLEIND.NS',
             'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS',
             'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TMPV.NS',
             'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS'],  # <-- edit universe
)

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s | %(levelname)s | %(message)s")


# ======================= TA HELPERS =======================

def rsi(series: pd.Series, length: int) -> pd.Series:
    delta = series.diff()
    gain = (delta.where(delta > 0, 0.0)).ewm(alpha=1/length, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0.0)).ewm(alpha=1/length, adjust=False).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


def atr(df: pd.DataFrame, length: int) -> pd.Series:
    high, low, close = df["High"], df["Low"], df["Close"]
    prev_close = close.shift(1)
    tr = pd.concat(
        [
            (high - low),
            (high - prev_close).abs(),
            (low - prev_close).abs(),
        ],
        axis=1,
    ).max(axis=1)
    return tr.rolling(length).mean()


def stoch_rsi(rsi_series: pd.Series, length: int) -> pd.Series:
    min_rsi = rsi_series.rolling(length).min()
    max_rsi = rsi_series.rolling(length).max()
    denom = (max_rsi - min_rsi)
    return (rsi_series - min_rsi) / denom.replace(0, np.nan) * 100.0


# ======================= GROWW INTRADAY COST MODEL =======================

def _groww_brokerage(turnover: float) -> float:
    base = min(20.0, 0.001 * turnover)
    if base < 5.0:
        base = min(5.0, 0.025 * turnover)
    return base


def groww_intraday_charges(buy_turnover: float, sell_turnover: float) -> dict:
    br_buy = _groww_brokerage(buy_turnover)
    br_sell = _groww_brokerage(sell_turnover)
    brokerage = br_buy + br_sell

    stt = 0.00025 * sell_turnover                # sell only
    stamp = 0.00003 * buy_turnover               # buy only
    exch_buy = 0.0000297 * buy_turnover
    exch_sell = 0.0000297 * sell_turnover
    exchange_fees = exch_buy + exch_sell
    sebi = 0.000001 * (buy_turnover + sell_turnover)
    ipft = 0.000001 * (buy_turnover + sell_turnover)
    gst_base = brokerage + exchange_fees + sebi + ipft
    gst = 0.18 * gst_base

    total = brokerage + stt + stamp + exchange_fees + sebi + ipft + gst

    return {
        "Brokerage": brokerage,
        "STT": stt,
        "StampDuty": stamp,
        "ExchangeFees": exchange_fees,
        "SEBI": sebi,
        "IPFT": ipft,
        "GST": gst,
        "TotalCharges": total,
    }


# ======================= RENKO =======================

def build_renko(close: pd.Series, brick_size: float) -> pd.DataFrame:
    if brick_size <= 0 or close.empty:
        return pd.DataFrame(columns=["Close", "Direction"])

    bricks_time, bricks_close, bricks_dir = [], [], []
    last_brick_price = close.iloc[0]
    last_dir = 0

    for ts, price in close.items():
        diff = price - last_brick_price
        while abs(diff) >= brick_size:
            direction = 1 if diff > 0 else -1
            if last_dir != 0 and direction != last_dir:
                last_brick_price += direction * brick_size
            else:
                last_brick_price += direction * brick_size

            bricks_time.append(ts)
            bricks_close.append(last_brick_price)
            bricks_dir.append(direction)
            last_dir = direction
            diff = price - last_brick_price

    if not bricks_time:
        return pd.DataFrame(columns=["Close", "Direction"])

    return pd.DataFrame(
        {"Close": bricks_close, "Direction": bricks_dir},
        index=pd.DatetimeIndex(bricks_time),
    )


def renko_support_resistance(renko: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    supports, resistances = [], []
    closes = renko["Close"].values
    times = renko.index

    for i in range(1, len(renko) - 1):
        prev_c, c, next_c = closes[i - 1], closes[i], closes[i + 1]
        ts = times[i]
        if c < prev_c and c < next_c:
            supports.append((ts, c))
        elif c > prev_c and c > next_c:
            resistances.append((ts, c))

    sup_df = (pd.DataFrame([{"Time": t, "Price": p} for t, p in supports])
              .set_index("Time")) if supports else pd.DataFrame(columns=["Price"])
    res_df = (pd.DataFrame([{"Time": t, "Price": p} for t, p in resistances])
              .set_index("Time")) if resistances else pd.DataFrame(columns=["Price"])
    return sup_df, res_df


def nearest_support(price: float, supports: pd.DataFrame,
                    brick_size: float, tol_frac: float) -> Optional[float]:
    if supports.empty:
        return None
    below = supports[supports["Price"] <= price]
    if below.empty:
        return None
    level = below["Price"].max()
    if price - level <= brick_size * tol_frac:
        return level
    return None


def nearest_resistance(price: float, resistances: pd.DataFrame,
                       brick_size: float, tol_frac: float) -> Optional[float]:
    if resistances.empty:
        return None
    above = resistances[resistances["Price"] >= price]
    if above.empty:
        return None
    level = above["Price"].min()
    if level - price <= brick_size * tol_frac:
        return level
    return None


# ======================= DATA & INDICATORS =======================

def yf_download(symbol: str, period: str, interval: str) -> pd.DataFrame:
    logging.info(f"Downloading {symbol} {interval} data ({period}) from yfinance...")
    df = yf.download(
        symbol,
        period=period,
        interval=interval,
        auto_adjust=False,
        progress=False,
        threads=False,
        multi_level_index=False,
    )

    if df.empty:
        logging.warning(f"No data for {symbol}.")
        return df

    df = df[["Open", "High", "Low", "Close", "Volume"]].copy()
    df.dropna(inplace=True)

    # Convert to IST
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC").tz_convert("Asia/Kolkata")
    else:
        df.index = df.index.tz_convert("Asia/Kolkata")

    return df


def prepare_indicators(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["RSI"] = rsi(df["Close"], CFG.rsi_len)
    df["ATR"] = atr(df, CFG.atr_len)
    df["StochRSI"] = stoch_rsi(df["RSI"], CFG.stoch_len)
    return df


# ======================= BACKTEST =======================

def backtest_symbol(symbol: str) -> pd.DataFrame:
    df = yf_download(symbol, CFG.period, CFG.interval)
    if df.empty or len(df) < 100:
        return pd.DataFrame()

    df = prepare_indicators(df)

    brick_size = df["ATR"].iloc[-1]
    if not (brick_size > 0):
        logging.warning(f"{symbol}: invalid brick size.")
        return pd.DataFrame()

    renko = build_renko(df["Close"], brick_size)
    if renko.empty:
        logging.warning(f"{symbol}: Renko bricks empty.")
        return pd.DataFrame()

    sup_df, res_df = renko_support_resistance(renko)

    # Precompute end-of-day flags (per bar)
    dates = df.index.date
    # last bar of day: date changes on next bar OR it's the last bar overall
    is_eod_bar = np.r_[dates[1:] != dates[:-1], True]

    trades = []
    in_pos = False
    pos_side = None
    entry_price = entry_time = None
    stop_price = target_price = None
    qty = 0
    capital = CFG.initial_capital

    idx = df.index

    for i in range(1, len(df) - 1):
        ts = idx[i]
        row = df.iloc[i]
        price = row["Close"]
        eod_here = bool(is_eod_bar[i])

        if not in_pos:
            if capital <= 0:
                continue

            # we DON'T open new trades on the last bar of the day
            if eod_here:
                continue

            # indicators & Renko S/R
            rsi_prev, rsi_now = df["RSI"].iloc[i - 1], df["RSI"].iloc[i]
            s_prev, s_now = df["StochRSI"].iloc[i - 1], df["StochRSI"].iloc[i]

            sup_level = nearest_support(price, sup_df, brick_size, CFG.sr_tolerance)
            res_level = nearest_resistance(price, res_df, brick_size, CFG.sr_tolerance)

            long_signal = (
                sup_level is not None and
                rsi_prev < 30 <= rsi_now and
                s_prev < 30 <= s_now
            )
            short_signal = (
                res_level is not None and
                rsi_prev > 70 >= rsi_now and
                s_prev > 70 >= s_now
            )

            if long_signal or short_signal:
                entry_idx = i + 1        # next 15m bar (same day)
                entry_row = df.iloc[entry_idx]
                entry_time = idx[entry_idx]
                entry_price = float(entry_row["Open"])

                # sizing with leverage
                max_exposure = capital * CFG.leverage
                max_shares = math.floor(max_exposure / entry_price)
                if max_shares <= 0:
                    continue

                qty = max_shares
                exposure = entry_price * qty
                capital_before = capital

                signal_row = row
                if long_signal:
                    pos_side = "long"
                    stop_price = float(signal_row["Low"])
                    target_price = res_level if res_level is not None else (
                        entry_price + 3 * brick_size
                    )
                else:
                    pos_side = "short"
                    stop_price = float(signal_row["High"])
                    target_price = sup_level if sup_level is not None else (
                        entry_price - 3 * brick_size
                    )

                in_pos = True

        else:
            # manage open position
            row = df.iloc[i]
            high, low = float(row["High"]), float(row["Low"])
            exit_reason = None
            exit_price = None

            if pos_side == "long":
                if low <= stop_price:
                    exit_price = stop_price
                    exit_reason = "SL"
                elif high >= target_price:
                    exit_price = target_price
                    exit_reason = "TP"
            else:
                if high >= stop_price:
                    exit_price = stop_price
                    exit_reason = "SL"
                elif low <= target_price:
                    exit_price = target_price
                    exit_reason = "TP"

            # HARD EOD SQUARE-OFF: if still in trade on last bar of the day,
            # exit at close of this bar
            if exit_reason is None and eod_here:
                exit_price = float(row["Close"])
                exit_reason = "EOD"

            if exit_reason is not None:
                exit_time = idx[i]

                buy_turnover = entry_price * qty
                sell_turnover = exit_price * qty

                if pos_side == "long":
                    gross_pnl = (exit_price - entry_price) * qty
                else:
                    gross_pnl = (entry_price - exit_price) * qty

                fees = groww_intraday_charges(buy_turnover, sell_turnover)
                charges = fees["TotalCharges"]
                net_pnl = gross_pnl - charges

                capital_after = capital_before + net_pnl
                capital = capital_after

                trades.append(
                    {
                        "Symbol": symbol,
                        "Side": pos_side,
                        "EntryTime": entry_time,   # IST
                        "EntryPrice": entry_price,
                        "ExitTime": exit_time,     # IST
                        "ExitPrice": exit_price,
                        "Qty": qty,
                        "Exposure": buy_turnover,
                        "BuyTurnover": buy_turnover,
                        "SellTurnover": sell_turnover,
                        "GrossPnL": gross_pnl,
                        "NetPnL": net_pnl,
                        "ReturnPctNet_on_Exposure": net_pnl / buy_turnover * 100.0,
                        "ReturnPctNet_on_Equity": net_pnl / capital_before * 100.0,
                        "ExitReason": exit_reason,
                        "CapitalBefore": capital_before,
                        "CapitalAfter": capital_after,
                        "Brokerage": fees["Brokerage"],
                        "STT": fees["STT"],
                        "StampDuty": fees["StampDuty"],
                        "ExchangeFees": fees["ExchangeFees"],
                        "SEBI": fees["SEBI"],
                        "IPFT": fees["IPFT"],
                        "GST": fees["GST"],
                        "TotalCharges": fees["TotalCharges"],
                    }
                )

                # reset
                in_pos = False
                pos_side = None
                entry_price = entry_time = None
                stop_price = target_price = None
                qty = 0

    if not trades:
        return pd.DataFrame()

    return pd.DataFrame(trades)


def print_cumulative_pnl(all_trades: pd.DataFrame) -> None:
    """Print cumulative profit and loss analysis"""
    if all_trades.empty:
        print("No trades to analyze for cumulative P&L")
        return
    
    # Sort trades by exit time to get chronological order
    trades_sorted = all_trades.sort_values('ExitTime').copy()
    
    # Calculate cumulative P&L
    trades_sorted['CumulativeNetPnL'] = trades_sorted['NetPnL'].cumsum()
    trades_sorted['CumulativeGrossPnL'] = trades_sorted['GrossPnL'].cumsum()
    
    # Calculate running totals by symbol
    symbol_cumulative = trades_sorted.groupby('Symbol')['NetPnL'].cumsum()
    
    print("\n" + "="*80)
    print("CUMULATIVE PROFIT & LOSS ANALYSIS")
    print("="*80)
    
    # Overall cumulative stats
    total_trades = len(trades_sorted)
    final_cumulative = trades_sorted['CumulativeNetPnL'].iloc[-1]
    final_gross_cumulative = trades_sorted['CumulativeGrossPnL'].iloc[-1]
    total_charges = trades_sorted['TotalCharges'].sum()
    
    print(f"\nOVERALL CUMULATIVE RESULTS:")
    print(f"Total Trades: {total_trades}")
    print(f"Final Cumulative Gross P&L: ₹{final_gross_cumulative:,.2f}")
    print(f"Total Charges: ₹{total_charges:,.2f}")
    print(f"Final Cumulative Net P&L: ₹{final_cumulative:,.2f}")
    
    # Peak and trough analysis
    peak_pnl = trades_sorted['CumulativeNetPnL'].max()
    trough_pnl = trades_sorted['CumulativeNetPnL'].min()
    max_drawdown = peak_pnl - trough_pnl
    
    print(f"\nDRAWDOWN ANALYSIS:")
    print(f"Peak Cumulative P&L: ₹{peak_pnl:,.2f}")
    print(f"Trough Cumulative P&L: ₹{trough_pnl:,.2f}")
    print(f"Maximum Drawdown: ₹{max_drawdown:,.2f}")
    
    # Symbol-wise cumulative P&L
    print(f"\nSYMBOL-WISE CUMULATIVE P&L:")
    symbol_summary = trades_sorted.groupby('Symbol').agg({
        'NetPnL': ['count', 'sum', 'mean'],
        'CumulativeNetPnL': 'last'
    }).round(2)
    symbol_summary.columns = ['Trades', 'Total_PnL', 'Avg_PnL', 'Final_Cumulative']
    print(symbol_summary.sort_values('Total_PnL', ascending=False))
    
    # Monthly cumulative breakdown
    trades_sorted['Month'] = trades_sorted['ExitTime'].dt.to_period('M')
    monthly_pnl = trades_sorted.groupby('Month')['NetPnL'].sum()
    monthly_cumulative = monthly_pnl.cumsum()
    
    print(f"\nMONTHLY CUMULATIVE P&L:")
    for month, cum_pnl in monthly_cumulative.items():
        monthly_pnl_val = monthly_pnl[month]
        print(f"{month}: Monthly ₹{monthly_pnl_val:,.2f} | Cumulative ₹{cum_pnl:,.2f}")
    
    # Win/Loss streaks
    trades_sorted['IsWin'] = trades_sorted['NetPnL'] > 0
    trades_sorted['Streak'] = trades_sorted['IsWin'].astype(int).diff().fillna(0).cumsum()
    
    win_streaks = []
    loss_streaks = []
    current_streak = 0
    current_type = None
    
    for _, trade in trades_sorted.iterrows():
        is_win = trade['IsWin']
        if current_type == is_win:
            current_streak += 1
        else:
            if current_type is not None:
                if current_type:
                    win_streaks.append(current_streak)
                else:
                    loss_streaks.append(current_streak)
            current_streak = 1
            current_type = is_win
    
    # Add the last streak
    if current_type is not None:
        if current_type:
            win_streaks.append(current_streak)
        else:
            loss_streaks.append(current_streak)
    
    print(f"\nSTREAK ANALYSIS:")
    if win_streaks:
        print(f"Longest Win Streak: {max(win_streaks)} trades")
        print(f"Average Win Streak: {np.mean(win_streaks):.1f} trades")
    if loss_streaks:
        print(f"Longest Loss Streak: {max(loss_streaks)} trades")
        print(f"Average Loss Streak: {np.mean(loss_streaks):.1f} trades")


def aggregate_results(all_trades: pd.DataFrame) -> None:
    if all_trades.empty:
        logging.warning("No trades generated.")
    else:
        total_net = all_trades["NetPnL"].sum()
        winrate = (all_trades["NetPnL"] > 0).mean() * 100.0

        logging.info(f"Total trades: {len(all_trades)}")
        logging.info(f"Winrate (net): {winrate:.2f}%")
        logging.info(f"Total Net P&L: ₹{total_net:,.2f}")

        summary = (
            all_trades.groupby("Symbol")["NetPnL"]
            .agg(["count", "sum", "mean"])
            .rename(columns={"count": "Trades", "sum": "NetPnL", "mean": "AvgNetPnL"})
        )
        print("\nPer-symbol summary (Net P&L):")
        print(summary)

        # Print cumulative P&L analysis
        print_cumulative_pnl(all_trades)

        all_trades.to_csv(CFG.out_trades_csv, index=False)
        logging.info(f"Trades saved to {CFG.out_trades_csv}")


def main():
    if not CFG.tickers:
        raise ValueError("Set CFG.tickers first")

    all_trades_list = []
    for symbol in CFG.tickers:
        trades = backtest_symbol(symbol)
        if not trades.empty:
            all_trades_list.append(trades)

    if not all_trades_list:
        logging.warning("No trades for any symbol.")
        return

    all_trades = pd.concat(all_trades_list, ignore_index=True)
    aggregate_results(all_trades)


if __name__ == "__main__":
    main()


2025-11-14 00:19:52,898 | INFO | Downloading ADANIENT.NS 15m data (60d) from yfinance...
2025-11-14 00:19:53,534 | INFO | Downloading ADANIPORTS.NS 15m data (60d) from yfinance...
2025-11-14 00:19:54,165 | INFO | Downloading APOLLOHOSP.NS 15m data (60d) from yfinance...
2025-11-14 00:19:54,751 | INFO | Downloading ASIANPAINT.NS 15m data (60d) from yfinance...
2025-11-14 00:19:55,353 | INFO | Downloading AXISBANK.NS 15m data (60d) from yfinance...
2025-11-14 00:19:55,890 | INFO | Downloading BAJAJ-AUTO.NS 15m data (60d) from yfinance...
2025-11-14 00:19:56,485 | INFO | Downloading BAJFINANCE.NS 15m data (60d) from yfinance...
2025-11-14 00:19:57,071 | INFO | Downloading BAJAJFINSV.NS 15m data (60d) from yfinance...
2025-11-14 00:19:57,666 | INFO | Downloading BEL.NS 15m data (60d) from yfinance...
2025-11-14 00:19:58,219 | INFO | Downloading BHARTIARTL.NS 15m data (60d) from yfinance...
2025-11-14 00:19:58,826 | INFO | Downloading CIPLA.NS 15m data (60d) from yfinance...
2025-11-14 00:1


Per-symbol summary (Net P&L):
               Trades        NetPnL    AvgNetPnL
Symbol                                          
ADANIENT.NS         6   4131.435775   688.572629
ADANIPORTS.NS       3   7977.030768  2659.010256
APOLLOHOSP.NS      11  -4722.412201  -429.310200
ASIANPAINT.NS       2  -4819.618338 -2409.809169
AXISBANK.NS         5  -2804.098876  -560.819775
BAJAJ-AUTO.NS       5   4238.095574   847.619115
BAJAJFINSV.NS       5  -3661.320026  -732.264005
BAJFINANCE.NS       5     93.420060    18.684012
BEL.NS              4  10487.007743  2621.751936
BHARTIARTL.NS       7  -8680.012357 -1240.001765
CIPLA.NS            9  -3505.777275  -389.530808
COALINDIA.NS        4   3366.382185   841.595546
DRREDDY.NS          5  -6533.497512 -1306.699502
EICHERMOT.NS        5  10235.595308  2047.119062
ETERNAL.NS          4  -8358.153744 -2089.538436
GRASIM.NS           5  -3223.352168  -644.670434
HCLTECH.NS          2  -3718.704639 -1859.352319
HDFCBANK.NS         1   2780.166449  2